In [1]:
from torchfsdd import TorchFSDDGenerator, TrimSilence
from torchaudio.transforms import MFCC
from torchvision.transforms import Compose

from matplotlib import pyplot as plt



# Create a transformation pipeline to apply to the recordings
transforms = Compose([
    TrimSilence(threshold=1e-6),
    MFCC(sample_rate=8e3, n_mfcc=13)
])

# Fetch the latest version of FSDD and initialize a generator with those files
fsdd = TorchFSDDGenerator(version='master', transforms=None)

# Create a Torch dataset for the entire dataset from the generator
full_set = fsdd.full()
# Create two Torch datasets for a train-test split from the generator
# train_set, test_set = fsdd.train_test_split(test_size=0.1)
# Create three Torch datasets for a train-validation-test split from the generator
train_set, val_set, test_set = fsdd.train_val_test_split(test_size=0.15, val_size=0.15)

In [8]:
data = None
for data in test_set:
    print(data)
    break

In [7]:
data

In [6]:
sample, label = data

f, ax = plt.subplots()
ax.imshow(sample.T, aspect='auto', origin='lower')
ax.set_title(f'Label: {label}')
ax.set_xlabel('Time (frames)')
ax.set_ylabel('MFCC Coefficients')
plt.show()

TypeError: cannot unpack non-iterable NoneType object